# Improving our log reg model for better improvement

So we saw even blending a log reg model initially at 0.849 with lasso model of 0.868 improves at 0.869! We will see if improving our log reg score will help towards that. 


Hence, let's try implementing bayesian methods to logistic regression. To do so we will try two different implementations - pymc3 and pystan.

# pystan

In [1]:
import sys, os
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, r2_score, make_scorer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import theano.tensor as t
from scipy.stats import mode
import pymc3 as pm

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

C:\Users\JoonH\AppData\Local\conda\conda\envs\TF\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import data
train = pd.read_csv("/Users/JoonH/dont-overfit-ii/train.csv")
train_y = train['target'].astype(int)
train_X = train.drop(['id','target'], axis=1).values

test_df = pd.read_csv("/Users/JoonH/dont-overfit-ii/test.csv")
test = test_df.drop(['id'], axis=1).values

# scale using RobustScaler
# fitting scaler on full data outperforms fitting on test_X only (+0.006 kaggle score)
data = RobustScaler().fit_transform(np.concatenate((train_X, test), axis=0))
train_X = data[:250]
test = data[250:]

In [3]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 200, n_jobs = 4, class_weight = 'balanced', max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators = 'auto', verbose = 0, max_iter = 5)
boruta_selector.fit(train_X,train_y)

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=489, n_jobs=4, oob_score=False,
            random_state=<mtrand.RandomState object at 0x000001FE5B484510>,
            verbose=0, warm_start=False),
     max_iter=5, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x000001FE5B484510>,
     two_step=True, verbose=0)

In [4]:
feature_df = pd.DataFrame(train.drop(['id','target'],axis=1).columns.tolist(),columns = ['features'])
feature_df['rank'] = boruta_selector.ranking_
feature_df = feature_df.sort_values('rank',ascending=True).reset_index(drop=True)
feature_df.head(3)

,features,rank
0,33,2
1,65,2
2,217,3


In [5]:
#keep top K features
K = 80
columns_to_keep = feature_df.features[0:K]

In [6]:
boruta_train = train[columns_to_keep]
boruta_test = test_df[columns_to_keep]
boruta_train.head()

,33,65,217,91,117,80,24,295,189,17,...,102,269,188,54,257,110,77,95,67,149
0,0.385,-0.770,1.187,0.019,0.710,1.851,1.763,-2.097,-0.443,-0.673,...,-0.704,-0.460,-0.317,-0.840,-1.053,-0.778,0.130,0.638,-1.504,-0.841
1,-2.721,1.221,0.216,1.188,0.987,-0.759,-1.519,-1.624,-1.178,-0.237,...,0.379,1.267,-0.354,0.051,-1.258,1.911,1.195,0.620,1.133,1.402
2,0.924,0.943,0.269,0.269,-0.384,0.758,1.786,-1.165,-0.702,0.270,...,0.651,-0.491,-0.829,-0.347,2.470,-0.199,-0.971,0.857,0.593,-0.191
3,0.394,-0.706,0.066,1.103,-0.152,0.030,0.365,0.467,-1.056,0.836,...,-0.418,0.769,-0.455,0.508,-0.591,-0.819,1.100,-1.108,1.629,0.817
4,0.037,0.357,0.110,0.892,1.027,-0.187,0.024,1.378,-0.602,-0.322,...,-0.617,-1.517,0.119,0.620,0.231,-0.364,0.350,0.114,-0.142,0.850


In [7]:
# scale using RobustScaler
# fitting scaler on full data outperforms fitting on test_X only (+0.006 kaggle score)
data = RobustScaler().fit_transform(np.concatenate((boruta_train, boruta_test), axis=0))
train_X = data[:250]
test = data[250:]
# add a bit of noise to train_X to reduce overfitting
train_X += np.random.normal(0, 0.01, train_X.shape)

In [8]:
import pystan

In [16]:
#experimentation

code = """                                                                                           
data {                                                                                               
  int N; //the number of training observations                                                       
  int N2; //the number of test observations                                                          
  int K; //the number of features                                                                    
  int y[N]; //the response                                                                           
  matrix[N,K] X; //the model matrix                                                                  
  matrix[N2,K] new_X; //the matrix for the predicted values                                          
}                                                                                                    
parameters {                                                                                         
  real alpha;                                                                                        
  vector[K] beta; //the regression parameters                                                        
}                                                                                                    
transformed parameters {                                                                             
  vector[N] linpred;                                                                                 
  linpred = alpha+X*beta;                                                                            
}                                                                                                    
model {                                                                                              
  alpha ~ cauchy(0,10); //prior for the intercept following Gelman 2008                              
                                                                                                     
  for(i in 1:K)                                                                                      
    beta[i] ~ student_t(1, 0, 0.035);                                                                 
                                                                                                     
  y ~ bernoulli_logit(linpred);                                                                      
}                                                                                                    
generated quantities {                                                                               
  vector[N2] y_pred;                                                                                 
  y_pred = alpha+new_X*beta; //the y values predicted by the model                                   
}                                                                                                    
"""    

data = {                                                                                             
    'N': 250,                                                                                        
    'N2': 19750,                                                                                     
    'K': 200,                                                                                        
    'y': train_y,                                                                                     
    'X': train_X,                                                                                      
    'new_X': test,                                                                                   
} 

n_itr = 3000
n_warmup = 1500

sm = pystan.StanModel(model_code = code)
fit = sm.sampling(data = data, iter = n_itr, warmup = n_warmup, seed = None)
ex = fit.extract(permuted = True)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1e1849401812d98f72a13a451f4daae7 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [17]:
import scipy
from scipy.stats import bernoulli

def logit_to_prob(logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return prob

#target = np.mean(ex['y_pred'], axis = 0)
target = np.mean(logit_to_prob(ex['y_pred']), axis = 0)
ids = test_df['id']
df = pd.DataFrame({'id': ids, 'target' : target})
df[['id', 'target']].to_csv("/Users/JoonH/DO2_pystan_log_student_0_0035.csv", index = False)

In [9]:
#https://www.kaggle.com/gkoundry/bayesian-logistic-regression-with-pystan                                                                                                     
code = """                                                                                           
data {                                                                                               
  int N; //the number of training observations                                                       
  int N2; //the number of test observations                                                          
  int K; //the number of features                                                                    
  int y[N]; //the response                                                                           
  matrix[N,K] X; //the model matrix                                                                  
  matrix[N2,K] new_X; //the matrix for the predicted values                                          
}                                                                                                    
parameters {                                                                                         
  real alpha;                                                                                        
  vector[K] beta; //the regression parameters                                                        
}                                                                                                    
transformed parameters {                                                                             
  vector[N] linpred;                                                                                 
  linpred = alpha+X*beta;                                                                            
}                                                                                                    
model {                                                                                              
  alpha ~ cauchy(0,10); //prior for the intercept following Gelman 2008                              
                                                                                                     
  for(i in 1:K)                                                                                      
    beta[i] ~ student_t(1, 0, 0.03);                                                                 
                                                                                                     
  y ~ bernoulli_logit(linpred);                                                                      
}                                                                                                    
generated quantities {                                                                               
  vector[N2] y_pred;                                                                                 
  y_pred = alpha+new_X*beta; //the y values predicted by the model                                   
}                                                                                                    
"""    

data = {                                                                                             
    'N': 250,                                                                                        
    'N2': 19750,                                                                                     
    'K': 80,                                                                                        
    'y': train_y,                                                                                     
    'X': train_X,                                                                                      
    'new_X': test,                                                                                   
} 

n_itr = 3000
n_warmup = 1000

sm = pystan.StanModel(model_code = code)
fit = sm.sampling(data = data, iter = n_itr, warmup = n_warmup, seed = None)
ex = fit.extract(permuted = True)

ValueError: Failed to parse Stan model 'anon_model_b03cb6e073632c0b3ebbf59424a12dda'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:

Probability function must end in _lpdf or _lpmf. Found distribution family = laplace with no corresponding probability function laplace_lpdf, laplace_lpmf, or laplace_log
  error in 'unknown file name' at line 22, column 35
  -------------------------------------------------
    20:                                                                                                      
    21:   for(i in 1:K)                                                                                      
    22:     beta[i] ~ laplace(1, 0, 0.03);                                                                 
                                          ^
    23:                                                                                                      
  -------------------------------------------------



In [64]:
import scipy
from scipy.stats import bernoulli

def logit_to_prob(logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return prob

In [66]:
#target = np.mean(ex['y_pred'], axis = 0)
target = np.mean(logit_to_prob(ex['y_pred']), axis = 0)
ids = test_df['id']
df = pd.DataFrame({'id': ids, 'target' : target})
df[['id', 'target']].to_csv("/Users/JoonH/DO2_pystan_log.csv", index = False)

This gives us LB score of 0.858, and blending this score with our lasso output we get 0.870!

*sources:*

1. https://www.kaggle.com/gkoundry/bayesian-logistic-regression-with-pystan/log
2. https://barnesanalytics.com/bayesian-logistic-regression-in-python-using-pymc3